# Problem 1: Data from yfinance

Importing the [yfinance](https://github.com/ranaroussi/yfinance) package.

In [2]:
import yfinance as yf

Getting each data in variables.

In [18]:
# yf.download('META', 'AAPL', 'AMZN', 'NFLX', 'GOOG', period='5d')
#meta_data = yf.download('META', period='5d')
# apple_data = yf.download('AAPL', period='5d')
# amazon_data = yf.download('AMZN', period='5d')
# netflix = yf.download('NFLX', period='5d')
# google_data = yf.download('GOOG', period='5d')

# Using `download` method to get data for 5 days period
# https://ranaroussi.github.io/yfinance/reference/api/yfinance.download.html#yfinance-download
yf.download(['META', 'AAPL', 'AMZN', 'NFLX', 'GOOG'], period='5d')

/tmp/ipykernel_7053/35421447.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  yf.download(['META', 'AAPL', 'AMZN', 'NFLX', 'GOOG'], period='5d')
[                       0%                       ]

[*********************100%***********************]  5 of 5 completed


Price            Close                                                   \
Ticker            AAPL        AMZN        GOOG        META         NFLX   
Date                                                                      
2025-09-25  256.869995  218.149994  246.570007  748.909973  1208.239990   
2025-09-26  255.460007  219.779999  247.179993  743.750000  1210.609985   
2025-09-29  254.429993  222.169998  244.360001  743.400024  1206.410034   
2025-09-30  254.630005  219.570007  243.550003  734.380005  1198.920044   
2025-10-01  256.399994  221.132401  244.369995  718.645020  1170.380127   

Price             High                                                   ...  \
Ticker            AAPL        AMZN        GOOG        META         NFLX  ...   
Date                                                                     ...   
2025-09-25  257.170013  220.669998  247.315002  756.770020  1216.839966  ...   
2025-09-26  257.600006  221.050003  250.119995  751.929993  1214.000000  ...   
2025-09-29  255.000000  222.600006  251.647995  750.780029  1224.489990  ...   
2025-09-30  255.919998  222.240005  243.675003  742.969971  1208.500000  ...   
2025-10-01  258.790009  221.490005  244.470001  721.849915  1179.000000  ...   

Price             Open                                                   \
Ticker            AAPL        AMZN        GOOG        META         NFLX   
Date                                                                      
2025-09-25  253.210007  220.059998  244.839996  753.450012  1203.089966   
2025-09-26  254.100006  219.080002  247.785004  750.000000  1205.420044   
2025-09-29  254.559998  220.080002  248.274994  748.719971  1205.000000   
2025-09-30  254.860001  222.029999  243.059998  742.250000  1206.410034   
2025-10-01  255.039993  217.259995  241.179993  721.489990  1175.119995   

Price         Volume                                         
Ticker          AAPL      AMZN      GOOG      META     NFLX  
Date                                                         
2025-09-25  55202100  52226300  17379800  10591100  1997800  
2025-09-26  46076300  41650100  16594600   9696300  1945000  
2025-09-29  40127700  44259200  23157200   9246800  3033200  
2025-09-30  37666900  48294000  22530900  16207600  3823900  
2025-10-01  22528783  21505380   6869735  12547328  2252842  

[5 rows x 25 columns]